# CSCA20: Lab 9, Week 10
## Databases

We will work with databases of type SQLite. Python has builtin methods to work with them as you saw in class. I will review the basic commands you will need to complete lab 9.

The first thing we want to do is connect to the database. If it doesn't exist this will create it.

In [11]:
import sqlite3

con = sqlite3.connect('planets.db')

# Now we make a cursor to connect with the data
cur = con.cursor()

# For now we will commit the changes (none)
con.commit()

# And close the cursor and the connection
cur.close()
con.close()

That is the basic syntax of how we will use the databse. Now we need to actually write or read data with it.

Let's look at some of the commands we can use:

To run a SQL command we use:

```Python
cur.execute('sql command')
```

If we want to create a new table we use:

```Python
cur.execute('''CREATE TABLE Precipitation(City TEXT, Snow REAL, Total INTEGER, Days INTEGER)''')
```

Here we create a table named Precipitation
The table has colums: City, Snow, Total, Days

And the types correspond to Python types as:
 - REAL -> float
 - TEXT -> str
 - INTEGER -> int


Some other useful commands are:

1. To delete (drop) a table:
```Python
cur.execute('DROP TABLE Precipitation')
```

2. Insert a row into the table:
```Python
cur.execute('''INSERT INTO Precipitation VALUES ("St. John\'s", 322.1, 1400, 200)''')
```
Alternatively:
```Python
cur.execute('''INSERT INTO Precipitation VALUES (?,?,?,?)''', ("St. John\'s", 322.1, 1400, 200))
```
Or:
```Python
city_name = "St. John\'s"
snow = 322.1
total = 1400
days = 200
cur.execute('''INSERT INTO Precipitation VALUES (?,?,?,?)''', (city_name, snow, total, days))
```

3. Fetch data from a table
```Python
# Select all (*) columns
cur.execute('SELECT * FROM Precipitation')
```
And then to use the data:
```Python
# Retrieve one line
line = cur.fetchone()
```
```Python
# Retrieve all lines
data = cur.fetchall()
```

```fetchone()``` and ```fetchall()``` return tuples containing the selected data.

Now let's try an example of using databases to store data and then read it back.

In [12]:
import sqlite3

def populate_planets(db, reader):
    """(str, reader) -> None
    Given the name of a database as a string and an
    open file, cereate the table planets and populate
    it with data.
    """
    # Open a connection with db
    con = sqlite3.connect(db)
    
    # Cursor to interact with the databse
    cur = con.cursor()
    
    # create table
    cur.execute('CREATE TABLE Planets(Name TEXT, Mass FLOAT, X FLOAT, Y FLOAT, Z FLOAT)')
    
    # Read the data and insert into the table line by line
    for line in reader:
        # My data is just divided by spaces so I just use split
        line = line.split()
       
        # Store the data
        name = line[0].strip()
        mass = float(line[1])
        x = float(line[2])
        y = float(line[3])
        z = float(line[4])
        
        # Insert into db
        cur.execute('''INSERT INTO Planets VALUES (?,?,?,?,?)''', (name, mass, x, y, z))
        
    # Commit the changes
    con.commit()
    
    # Close connection
    cur.close()
    con.close()
    

In [13]:
# Program to try the function
FILENAME = "solar_sys.txt"
DB_NAME = "Planets.db"

file = open(FILENAME, 'r')

populate_planets(DB_NAME, file)

file.close()

Now the databse should contain the data. Note that a db is not a text file so you cannot just "open" it in a text editr and see the data. We will read it back in in Python to see it. 

Let's try reading back just the mass and name

In [14]:
import sqlite3

con = sqlite3.connect("Planets.db")

cur = con.cursor()

cur.execute('''SELECT Name, Mass FROM Planets''')

for line in cur:
    print(line)
    
cur.close()
con.close()

('sun', 1.988544e+30)
('mercury', 3.302e+23)
('venus', 4.8685e+24)
('earth', 5.97219e+24)
('mars', 6.4185e+23)
('jupiter', 1.89813e+27)
('saturn', 5.68319e+26)
('uranus', 8.68103e+25)
('neptune', 1.0241e+26)


We can also select only a subset of items by defining a condition. These conditions are similar to python conditions:

In [15]:
import sqlite3

con = sqlite3.connect("Planets.db")

cur = con.cursor()

cur.execute('''SELECT Name, Mass FROM Planets WHERE Name="sun"''')

for line in cur:
    print(line)
    
cur.close()
con.close()

('sun', 1.988544e+30)


In [16]:
import sqlite3

con = sqlite3.connect("Planets.db")

cur = con.cursor()

cur.execute('''SELECT Name, Mass FROM Planets WHERE mass<"1e25"''')

for line in cur:
    print(line)
    
cur.close()
con.close()

('mercury', 3.302e+23)
('venus', 4.8685e+24)
('earth', 5.97219e+24)
('mars', 6.4185e+23)


We can also delete items in a similar way:

In [17]:
import sqlite3

con = sqlite3.connect("Planets.db")

cur = con.cursor()

cur.execute('''DELETE FROM Planets WHERE mass<"1e25"''')

con.commit()
    
cur.close()
con.close()

Now look at the db:

In [27]:
import sqlite3

con = sqlite3.connect("Planets.db")

cur = con.cursor()

cur.execute('''SELECT * FROM Planets''')

data = cur.fetchall()

print(data)

print("")

for item in data:
    print(item)


cur.close()
con.close()

[('sun', 1.988544e+30, 49140.3347836458, -363271.5592552171, -10491.48558556447), ('jupiter', 1.89813e+27, -777841420.2838075, 224451869.927134, 16474417.32604697), ('saturn', 5.68319e+26, -281880540.0917871, 1321479657.136385, -11774698.69993168), ('uranus', 8.68103e+25, 2668275057.715974, -1368207197.729832, -39654566.07834578), ('neptune', 1.0241e+26, 3068964883.54269, -3290508769.241064, -2965492.893564129)]

('sun', 1.988544e+30, 49140.3347836458, -363271.5592552171, -10491.48558556447)
('jupiter', 1.89813e+27, -777841420.2838075, 224451869.927134, 16474417.32604697)
('saturn', 5.68319e+26, -281880540.0917871, 1321479657.136385, -11774698.69993168)
('uranus', 8.68103e+25, 2668275057.715974, -1368207197.729832, -39654566.07834578)
('neptune', 1.0241e+26, 3068964883.54269, -3290508769.241064, -2965492.893564129)
